# Octave Band Analysis from XLSX

### Octave Filter Function 

In [32]:
import numpy as np
from scipy import signal
##import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import butter, sosfilt, filtfilt, resample

def ovtavefilter (filepath, target_sample_rate=22000) :
    """
    Filter a .wav signal with octave or fractional octave filter bank. 
    This function uses a Butterworth filter with default order of 6 and Second-Order Sections
    coefficients. To obtain the correct coefficients, a subsampling is
    applied to the signal in each filtered band.

    :param filepath: filepath to .wav signal
    :param target_sample_rate: Downsampling rate of 22 kHz
    :param fraction: Bandwidth 'b'. Examples: 1/3-octave b=3, 1-octave b=1,
    2/3-octave b = 3/2. [Optional] Default: 1.

    RETURN: Sound Pressure Level of each band in array variable
    
    """
    # load .wav file
    sample_rate, data = wavfile.read(filepath)

    # Umwandeln in Mono, falls Stereo vorhanden
    if len(data.shape) == 2:
        x = np.mean(data, axis=1)

    # Downsampling, falls erforderlich
    if sample_rate != target_sample_rate:
        downsample_factor = np.round(sample_rate / target_sample_rate)
        data = data[::int(downsample_factor)]
        sample_rate = target_sample_rate

    # Normalisierung
    data_mean = np.mean(data)
    data_std = np.std(data)
    if data_std > 0:
        data = (data - data_mean) / data_std

    # define frequency borders
    f_centers = [31.25, 62.5, 125, 250, 500, 1000, 2000, 4000, 8000, 16000] # Middle Border Frequencies
    f_lower = [f / np.sqrt(2) for f in f_centers] # Downside Border Frequencies
    f_upper = [f * np.sqrt(2) for f in f_centers] # Upside Border Frequencies

    # define output variable
    db_values = []  
    
    # Berechnung der Filter und Anwendung
    for low, high in zip(f_lower, f_upper):
        # Berechnung des Nyquist
        nyquist = sample_rate / 2

        # Berechnung der Normalisierten Frequenzen
        low_norm = low / nyquist
        high_norm = high / nyquist

        # Überprüfen, ob die Frequenzen im Bereich [0, 1] liegen
        if low_norm <= 0 or high_norm >= 1:
            print(f"Warnung: Frequenzen für Band {low} Hz - {high} Hz sind außerhalb des gültigen Bereichs.")
            continue  # Überspringen, wenn Frequenzen ungültig sind

        # Butterworth-Filter der Ordnung 6 als SOS-Filter designen
        sos = butter(6, [low_norm, high_norm], btype='band', output='sos')

        # Anwenden des Filters
        filtered_signal = sosfilt(sos, data)

        # Berechnung der dB-Werte
        rms = np.sqrt(np.mean(filtered_signal**2))
        db = 20 * np.log10(rms) if rms > 0 else -np.inf
        db_values.append(db)

    return np.array(db_values)


path = "/Users/Anton/Documents/KIT/Seminare/RTN/Audio rtn-samples/IDMT_Traffic/audio/2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_41346_M_D_CR_SE_CH34.wav"
#path = "/Users/Anton/Documents/KIT/Seminare/RTN/Audio rtn-samples/IDMT_Traffic/audio/2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_63533_M_D_CL_ME_CH12.wav"

octave_band_values_db = ovtavefilter(path)
print(octave_band_values_db)

Warnung: Frequenzen für Band 5656.8542494923795 Hz - 11313.70849898476 Hz sind außerhalb des gültigen Bereichs.
Warnung: Frequenzen für Band 11313.708498984759 Hz - 22627.41699796952 Hz sind außerhalb des gültigen Bereichs.
[-282.73724797 -246.73643948 -210.85941317 -175.23296593 -140.12011071
 -106.08851907  -74.49561031  -50.12823584]


### filter response visualisation

### Octave Band Analysis Function

In [ ]:
def process_audio_directory_OctaveBands(input_directory, output_file):
    """
    Calculates Octave Filter dB values for all .wav files in a directory and safes them as indivduals columns in an Excel-File. 
    Args:
        input_directory (str): location of .wav files
        output_file (str): output location / file
        
    """